# SIMPSONS DATASET

https://www.kaggle.com/alexattia/the-simpsons-characters-dataset/home

In [1]:
!ls ../datasets/the-simpsons-characters-dataset/simpsons_dataset/

ls: cannot access '../datasets/the-simpsons-characters-dataset/simpsons_dataset/': No such file or directory


#### import libraries

In [5]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.activations import elu
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam 


import os
import pickle
import cv2
import random 
import imutils
import numpy as np
from imutils import paths

### BUILD MODEL

In [7]:
class SmallVGGNet:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (width, height, depth)
        chanDim = -1

        # if we are using "channel first", update the input and channel dimension
        if K.image_data_format() == "channels_first":
            inputShape(depth, width, height)
            changDim=1

        # CONV -> RELU -> POOL

        model.add(Conv2D(32, (3,3), padding='same', input_shape=inputShape))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(3,3)))
        model.add(Dropout(0.25))

        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(64, (3,3), padding="same"))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3,3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.25))


        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(128, (3,3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3,3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.25))

        # first (and only) set of FC => Relu layers
        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        return model
        

#### Initialize configuration parameters 

In [8]:
#### initial configuration parameters
EPOCHS = 10
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (96,96,3)

#### INITIALIZE DATASET

In [13]:
print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images('../datasets/the-simpsons-characters-dataset/simpsons_dataset/')))
random.seed(42)
random.shuffle(imagePaths)

[INFO] loading images...


#### initialize data and labels 

In [17]:
data = []
labels = []

#### loop over input images 

In [18]:
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = img_to_array(image)
    data.append(image)
    
    # extract the class label from the image path and update the labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

In [19]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
print("[INFO] data matrix: {:.2f}MB".format(data.nbytes / (1024 * 1000.0)))

[INFO] data matrix: 9043.06MB


#### BINARIZE THE LABELS

In [20]:
lb = LabelBinarizerarizer()
labels = lb.fit_transform(labels)

## PARTITION INTO TRAIN TEST AND VALIDATION 80-20 

In [21]:
(train_X, val_X, train_Y, val_Y) = train_test_splitest_split(data, label, test_size=0.2,
                                                   random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [41866, 13]

### CONSTRUCT THE IMAGE GENERATOR FOR DATA AUGMENTATION 

In [ ]:
aug = ImageDataGenerator(rotation_range=25, width_shaift_range=0.1,
                        height_shift_range=0.1, shear_range=0.2,
                        horizontal_flip=True, fill_mode="nearest")

### INITIALIZE MODEL

In [ ]:
model = SmallVGGNet.build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],
                         depth=IMAGE_DIMS[2], classes=len(lb.classes_))
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimzer=opt, metrics=["accuracy"])